In [1]:
#Generally usefull functions

import random

# Function for querying the servers, randomly adds byz_num many errors
# in the response.

def query_byz(db, Queries, e, byz_num, file):
    len_row = n
    len_col = file_num*L
    E = matrix(F, len_col, len_row, 0)
    if L == 1:
        E[file] = e
    else:
        for l in range(L):
            E[file*L + l] = e[l]
    E = Queries + E
    byz_servers = random.sample(range(0, len_row), byz_num)
    response = vector(F, [0]*len_row)
    for i in range(len_row):
        r = 0 
        for j in range(len_col):
            r += E[j,i]*db[j,i]
        if (i in byz_servers):
            r += F.random_element()
        response[i] = r
    return response

# Returns an invertible submatrix of A and the correspoding 
# incidence vector.

def invertible_submatrix(A):
    lng = A.ncols()
    shrt = A.nrows()
    ret_mat = matrix(F, shrt, shrt, 0)
    ret_vec = vector(F, [0]*lng)
    for i in range(shrt):
        for j in range(lng):
            if (A[i,j] != 0):
                ret_mat[:,i] = A[:,j]
                ret_vec[j] = 1
                break
    if (ret_mat.rank() == shrt):
        return ret_mat, ret_vec 
    else:
        print("No invertible submatrix found")
        return ret_mat, ret_vec

# Star product of two vectors:

def star(a, b): 
    return a.pairwise_product(b)

# evaluation vector eval, such that eval_i = f(a_i)

def evaluate(f, a):
    le = a.length()
    ret = vector(F, [0]*le)
    for i in range(le):
        ret[i] = f(a[i])
    return ret

def polyencode(points):
    ret  = x^0-1
    for i in range(points.length()):
        ret += points[i]*x^i
    return ret


In [57]:
# Reed-Muller version:
# No byzantine/unresponsive servers

# Parameters:

m = 5
r_C = 1
r_D = 2
file_num = 5
n = 2^m
k_C = sum(binomial(m,i) for i in range(0,r_C+1))
k_D = sum(binomial(m,i) for i in range(0,r_D+1))
t = 2^(r_D+1)
print("This scheme protects against {}-collusion".format(t))
print("and allows the user to download {} symbols per round".format(sum(binomial(m,i) for i in range(0,m-r_C-r_D))))
alpha = AffineGroup(m,F)
L = 1
S = 1

This scheme protects against 8-collusion
and allows the user to download 6 symbols per round


In [58]:
# Setup:

F = GF(2)
C = codes.BinaryReedMullerCode(r_C,m)
G = C.generator_matrix()
print("The generator matrix of C: \n{}".format(G))
D = codes.BinaryReedMullerCode(r_D,m)
CstarD = codes.BinaryReedMullerCode(r_C+r_D, m)
CstarDperp = codes.BinaryReedMullerCode(m-r_C-r_D-1,m)
H = CstarDperp.generator_matrix()
print("Generator of (C*D)^perp, (H):\n{}".format(H))
#print("GG: \n{}".format(GG))

The generator matrix of C: 
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Generator of (C*D)^perp, (H):
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [59]:
# Setting up random files and the storage:
V = VectorSpace(F, k_C)
X = matrix(F,file_num,k_C,0)
for i in range(file_num):
     X[i] = V.random_element()
print("File system, (X): \n{}".format(X))
Y = X*G
print("Storage, (Y): \n{}".format(Y))


File system, (X): 
[1 1 1 1 1 1]
[0 0 1 1 0 1]
[0 0 0 0 1 0]
[1 1 1 0 0 1]
[1 1 0 1 1 0]
Storage, (Y): 
[1 0 0 1 0 1 1 0 0 1 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0 0 1 0 1 1 0]
[0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]
[1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0]
[1 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0]


In [60]:
# Generating random queries
Q = matrix(F, file_num, n, 0)
if (r_D == 0):
    for i in range(file_num):
        Q[i] = [1]*n
else:
    for i in range(file_num):
        Q[i] = D.random_element()
print("Random queries form D, (Q):\n{}".format(Q))

# Invertible submatrix of H
invert, incidence = invertible_submatrix(H)
print("\nInvertible submatrix of H and the corresponding incidence vector:\n{}\n\ne = {}\n".format(invert, 
                                                                                         incidence))
#-----------------------------------------
kk =CstarDperp.dimension()
ebin = matrix(F, kk, kk, 0)
ebin[0] = invert[0]
for i in range(1,kk):
    ebin[i] = invert[0] + invert[i]
#-----------------------------------------
# generating the decoding matrix
nums = list(range(1,k_C+1)) 
str_nums = map(str,nums)
xs = map(lambda x: "x"+x, str_nums)
variables = list(xs)
R = PolynomialRing(F,k_C,variables)
x = vector(R,variables)
#decoding_matrix = matrix(F, k_C, k_C, 0)
#for i in range(k_C):
#    for j in range (k_C):
#        decoding_matrix[i,j] = ((x*G).pairwise_product(incidence)*H.transpose())[i].coefficient({x[j]:1})
decoding_matrix = ebin #= decoding_matrix.inverse()
print("System of equations to solve for the file: \n{}".format((x*G).pairwise_product(incidence)*H.transpose()))
print("Decoding matrix (inverse of the equation matrix): \n{}".format(decoding_matrix))

Random queries form D, (Q):
[1 1 1 1 0 0 0 0 0 1 1 0 0 1 1 0 0 1 0 1 1 0 1 0 1 1 0 0 1 1 0 0]
[0 1 0 0 1 0 0 0 1 1 1 0 0 0 1 0 1 0 1 1 1 0 0 0 1 1 1 0 1 1 0 1]
[0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 1]
[1 1 0 0 0 1 1 0 1 0 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0]
[0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 0 0 0 1 1 0]

Invertible submatrix of H and the corresponding incidence vector:
[1 1 1 1 1 1]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]

e = (1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

System of equations to solve for the file: 
(x2 + x3 + x4 + x5 + x6, x1 + x2, x1 + x3, x1 + x4, x1 + x5, x1 + x6)
Decoding matrix (inverse of the equation matrix): 
[1 1 1 1 1 1]
[1 0 1 1 1 1]
[1 1 0 1 1 1]
[1 1 1 0 1 1]
[1 1 1 1 0 1]
[1 1 1 1 1 0]


In [61]:
# responses:
# index of the desired file:
# Let us test this over all files:

for index in range(file_num):
    print("Desired file index: {}\nand the file itself: {} ".format(index, X[index]))
    r = query_byz(Y, Q, incidence, 0, index)
    print("Total response vector, (r):\n{}".format(r))
    # project and invert
    result = decoding_matrix*H*r
    print("Result (Projection and inversion): \n{}".format(result))
    if result[0:6] == X[index]:
        print("CORRECT\n")
    else:
        print("FALSE\n")

Desired file index: 0
and the file itself: (1, 1, 1, 1, 1, 1) 
Total response vector, (r):
(1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0)
Result (Projection and inversion): 
(1, 1, 1, 1, 1, 1)
CORRECT

Desired file index: 1
and the file itself: (0, 0, 1, 1, 0, 1) 
Total response vector, (r):
(0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0)
Result (Projection and inversion): 
(0, 0, 1, 1, 0, 1)
CORRECT

Desired file index: 2
and the file itself: (0, 0, 0, 0, 1, 0) 
Total response vector, (r):
(0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0)
Result (Projection and inversion): 
(0, 0, 0, 0, 1, 0)
CORRECT

Desired file index: 3
and the file itself: (1, 1, 1, 0, 0, 1) 
Total response vector, (r):
(1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0)
Result (Projection and inversion): 
(1, 1, 1, 0, 0, 1)
C

In [7]:
# data import
F = GF(2^8)
maxlen = 0
row_num = 0

with open("ian2.dat") as f:
    for line in f:
        line = line.strip()
        l = len(list(map(lambda c: F(Integer(ord(c)).bits()), line)))
        if l > maxlen:
            maxlen = l
        row_num += 1
row_num

12557

In [17]:
#F = GF(2^8)
#maxlen = 86
#row_num = 100
L = 2
k = 43
X = matrix(F, L*row_num, k, 0)
        
with open("ian2.dat") as f:
    for i,line in enumerate(f):
        if i >= row_num:
            break
        line = line.strip()
        vec = list(map(lambda c: F(Integer(ord(c)).bits()), line))
        l = len(vec)
        if l < maxlen:
            vec = vec + ([0]*(maxlen-l))
        
        for j in range(L):
            X[i*L+j] = vector(F, vec[j*k:(j+1)*k])


In [11]:
#Reed-Solomon (Byzantine):

#Parameters:
F = GF(13)
n = 12
k = 4
t = 2
b = 2
r = 1
#file_num = row_num
file_num = 3
rho = n-(k+t+2*b+r-1)
#S = ceil(L*k / rho)
S = LCM(rho, k) / rho
L = LCM(rho, k) / k
alpha = F.list()[:n]
alpha_vec = vector(F, alpha)
beta_C = vector(F, [1]*n)
beta_D = vector(F, [1]*n)

#Codes:
C = codes.GeneralizedReedSolomonCode(alpha, k, beta_C)
D = codes.GeneralizedReedSolomonCode(alpha, t, beta_D)
CstarD = codes.GeneralizedReedSolomonCode(alpha, min(n, k+t-1), beta_C.pairwise_product(beta_D))
G = C.generator_matrix()
CorrectingCode = codes.GeneralizedReedSolomonCode(alpha, n-2*b-r, beta_C)

print("This scheme protects against {} collusions, {} errors and {} unresponsive servers.". format(t,b,r))
print("We retrieve {} symbols per rounds and the whole file in {} rounds.".format(rho, S))

#Setup:
X = matrix(F, L*file_num, k, 0)
V = VectorSpace(F, k)
for i in range(0, L*file_num):
    X[i] = V.random_element()
Y = X*G
print("FILES:\n{}".format(X))
print("STORAGE GENERATOR:\n{}".format(G))
print("SERVERS:\n{}".format(Y))


This scheme protects against 2 collusions, 2 errors and 1 unresponsive servers.
We retrieve 2 symbols per rounds and the whole file in 2 rounds.
FILES:
[ 0  9  6  2]
[ 9 11  1  5]
[10  6  2 10]
STORAGE GENERATOR:
[ 1  1  1  1  1  1  1  1  1  1  1  1]
[ 0  1  2  3  4  5  6  7  8  9 10 11]
[ 0  1  4  9  3 12 10 10 12  3  9  4]
[ 0  1  8  1 12  8  8  5  5  1 12  5]
SERVERS:
[ 0  4  6  5  0  3  0  3 11 10 12  3]
[ 9  0 10  4 12 12  8  4  4 12  6  3]
[10  2  6  4  4  1  3  5  2  2  0  4]


In [13]:
#Queries:
Q = matrix(F, L*file_num, n, 0)

#Iterating over S rounds:

print("Desired coefficients will be on terms of degree between {} and {}.".format(k+t-1, n-2*b-r-1))

R.<x> = F[]
index = 1
h = matrix(F, S, rho, 0)

print("\nBEGINNING QUERYING:\n")
for s in range(S):
    block = s*L
    subs = vector(F, [0]*n)
    #print("\n")
    # random queries for this round
    for i in range(0, L*file_num):
        Q[i] = D.random_element()
            
    # generating the retrieval vector
    e = matrix(F, L, n, 0)
    for l in range(L):
        p = (s+1)*rho + (-l)*k + t - 1
        if p >= 0:
            e[l] = evaluate(x^p, alpha_vec)
    
    #print("The evaluation matrix (e): \n{}".format(e))
        
    # polynomial for removing already known information
    if (s > 0):
        for sigma in range(s):
            exponent = rho*(s-sigma) + k + t - 1
            subs = subs + evaluate((x^exponent)*(polyencode(vector(F,h[sigma].list()))), alpha_vec)
        #print("Substracting what we know: {}".format(subs))
             
    # Querying the servers
    ret0 = query_byz(Y,Q,e,b,index) - subs
    #print("Resonse for this round: {}".format(ret0))
        
    # Error-correction:
    ret1 = CorrectingCode.decode_to_code(ret0) 
    #print("Error-corrected: {}".format(ret1))
        
    # Decoding:
    points = list(zip(alpha, ret1))
    poly = R.lagrange_polynomial(points)
    print("POLYNOMIAL RECIEVED BY INTERPOLATING: \n{}".format(poly))
    coeffs = poly.list()[k+t-1:n-2*b-r+1]
    if len(coeffs) < rho:
        zero_list = [0]*(rho-len(coeffs))
        coeffs = coeffs + zero_list
    #print("AND THE CORRESPONDING COEFFS: \n{}".format(coeffs))
    h[s] = vector(F,coeffs)
        
print("\nQUERYING FINISHED, CONSTRUCTING FINAL ANSWER:\n")
ans = matrix(F, 1, L*k, 0)
for i in range (S):
    for j in range(rho):
        ans[0,i*rho+j]  = h[i,rho-j-1]
answer = matrix(F, L, k, ans.list()) 
for i in range(L):
    answer[i] = answer[i][::-1]
print("FINAL RESULT: \n{}".format(answer))
lst = list(range(index*L,index*L+L))
print("\nWHAT IT SHOULD BE: \n{}".format(X.matrix_from_rows(lst)))
print("CORRECT ANSWER?")
answer == X.matrix_from_rows(lst)

Desired coefficients will be on terms of degree between 5 and 6.

BEGINNING QUERYING:

POLYNOMIAL RECIEVED BY INTERPOLATING: 
5*x^6 + x^5 + 10*x^4 + 9*x^3 + 7*x^2 + 6*x + 10
POLYNOMIAL RECIEVED BY INTERPOLATING: 
11*x^6 + 9*x^5 + 3*x^4 + 5*x^3 + 11*x^2 + 11*x + 2

QUERYING FINISHED, CONSTRUCTING FINAL ANSWER:

FINAL RESULT: 
[ 9 11  1  5]

WHAT IT SHOULD BE: 
[ 9 11  1  5]
CORRECT ANSWER?


True

In [22]:
# converting back to string
"".join(list(map(lambda f: chr(f.integer_representation()), answer.list()))).rstrip("\0x00")

'1600343060.122756,21.3 C,37.7 %,113.23,0,106.97,-327.7,-328.0,-328.0,-328.0,20.5'

In [15]:
# Lagrange encoded private polynomial computation:

# parameters:
F = GF(23); N = 10 ; K =3 ;E =1 ;T =1 ;G =3; P = 0; A = 0; M = 3; R = PolynomialRing(F, 'x');
if N <= G*(K+E-1)+T+P+2*A:
    N = G*(K+E-1)+T+P+2*A + 2
    
alpha = F.list()[1:N+1]
beta = F.list()[N+1:N+K+E+1]

V = VectorSpace(F, K+E)
X = matrix(F, M, K+E, 0)
for i in range(M):
    X[i] = V.random_element()

u = []
for i in range(M):
    points = list(zip(beta, list(X[i])))
    poly = R.lagrange_polynomial(points)
    u.append(poly)
    
R = LaurentPolynomialRing(F, 'x')
Y = matrix(F, M, N, 0)
for i in range(M):
    for j in range(N):
        Y[i,j] = u[i](alpha[j])

# more parameters
NN = N - (P + 2*A)
H = NN -(G*(K+E-1)+T)
L = G*(K+E-1) + 1
B = lcm(L,H) / L
S = lcm(L,H) / H
# phi is the collection of functions, we want to evaluate
phi = []
for i in range(B):
    phi.append(x)

zeta = matrix(R, S, B, 0)
Q = matrix(F,1,S,0)
U = matrix(F,1,S,0)
Q[0,0] = H//L
U[0,0] = H%L
for b in range(B):
    if b >= 0 and b<=Q[0,0]:
        zeta[0,b] = x^(b*L)
    else:
        zeta[0,b] = 0
        
rho = matrix(R,1,S,0)
for s in range(S):
    if s == 0:
        psi = vector(R,[0]*T)
        for t in range(T):
            psi[t] = F.random_element()
            
        
        for b in range(B):
            rho[0,s] += zeta[s,b]*phi[b]
        for t in range(T):
            rho[0,s] += x^(H+t)*psi[t]
        
        response = matrix(R, M, N, 0)
        # query function:
        for i in range(N):
            for j in range(M):
                response[i,j] = rho[0,s](Y[j,i])
                
        
    else:
        Qs = 1
        while((Qs-Q[0,s-1])*L-U[0,s-1] <= H):
            Qs += 1
        
        Q[0,s] = Qs-1
        U[0,s] = H + U[0,s-1] - (Q[0,s]-Q[0,s-1])*L
        
        for b in range(B):
            if (Q[0,s-1] <= b and b <= Q[0,s]):
                exp = -U[0,s-1]+(b-(Q[0,s-1]+1))*L
                zeta[s,b] = x^exp
            else:
                zeta[s,b] = 0
                
        psi = vector(R,[0]*T)
        for t in range(T):
            psi[t] = R.random_element()
            while(psi[t].degree() > G):
                psi[t] = R.random_element()
        
        for b in range(B):
            rho[0,s] += zeta[s,b]*phi[b]
        for t in range(T):
            rho[0,s] += x^(H+t)*psi[t]
        
        response = matrix(R, M, N, 0)
        # query function:
        for i in range(N):
            for j in range(M):
                response[i,j] = rho[0,s](Y[j,i])

TypeError: positive characteristic not allowed in symbolic computations